In [16]:
import gzip
import math
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from collections import defaultdict
import numpy as np

In [3]:
def readGz(path):
    for l in gzip.open(path, 'rt', encoding='cp437', errors='ignore'):
        yield eval(l)

In [4]:
dataset = []
for l in readGz("australian_users_items.json.gz"):
    dataset.append(l)

In [5]:
pairs = []
for d in dataset:
    for i in d['items']:
        pairs.append((d['user_id'], i['item_id'], i['playtime_forever']))

In [6]:
itemsPerUser = defaultdict(set)
usersPerItem = defaultdict(set)

for p in pairs:
    u = p[0]
    i = p[1]
    t = p[2]
    if t > 0:
        itemsPerUser[u].add(i)
        usersPerItem[i].add(u)

In [7]:
len(itemsPerUser)
# number of unique users in the dataset

68403

In [8]:
len(usersPerItem)
# number of unique games in the dataset

10050

In [9]:
len(dataset)

88310

In [12]:
count = 0
for d in dataset:
    if len(d["items"]) == 0 :
        count += 1
count

16806

In [21]:
userId = [d["steam_id"] for d in dataset]

In [22]:
userId = np.array(userId)

In [25]:
unique, counts = np.unique(userId, return_counts=True)

In [37]:
# read in meta data for games
datasetGames = []
for l in readGz("steam_games.json.gz"):
    datasetGames.append(l)

In [40]:
datasetGames[0]

{'publisher': 'Kotoshiro',
 'genres': ['Action', 'Casual', 'Indie', 'Simulation', 'Strategy'],
 'app_name': 'Lost Summoner Kitty',
 'title': 'Lost Summoner Kitty',
 'url': 'http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/',
 'release_date': '2018-01-04',
 'tags': ['Strategy', 'Action', 'Indie', 'Casual', 'Simulation'],
 'discount_price': 4.49,
 'reviews_url': 'http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1',
 'specs': ['Single-player'],
 'price': 4.99,
 'early_access': False,
 'id': '761140',
 'developer': 'Kotoshiro'}

In [43]:
gameGenres = {}
for game in datasetGames:
    print(game)
    if game["title"] in gameGenres.keys():
        print(game["title"])
    else:
        gameGenres[game["title"]] = game["genres"]
    

{'publisher': 'Kotoshiro', 'genres': ['Action', 'Casual', 'Indie', 'Simulation', 'Strategy'], 'app_name': 'Lost Summoner Kitty', 'title': 'Lost Summoner Kitty', 'url': 'http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/', 'release_date': '2018-01-04', 'tags': ['Strategy', 'Action', 'Indie', 'Casual', 'Simulation'], 'discount_price': 4.49, 'reviews_url': 'http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1', 'specs': ['Single-player'], 'price': 4.99, 'early_access': False, 'id': '761140', 'developer': 'Kotoshiro'}
{'publisher': 'Making Fun, Inc.', 'genres': ['Free to Play', 'Indie', 'RPG', 'Strategy'], 'app_name': 'Ironbound', 'sentiment': 'Mostly Positive', 'title': 'Ironbound', 'url': 'http://store.steampowered.com/app/643980/Ironbound/', 'release_date': '2018-01-04', 'tags': ['Free to Play', 'Strategy', 'Indie', 'RPG', 'Card Game', 'Trading Card Game', 'Turn-Based', 'Fantasy', 'Tactical', 'Dark Fantasy', 'Board Game', 'PvP', '2D', 'Competitive', 'Repl

KeyError: 'title'